<a href="https://colab.research.google.com/github/SaShakib/EEG_research/blob/main/EEG_Training.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!ls "/content/drive/My Drive/Shakib(10sec_each)"

shakib_Backward_s10.json  shakib_Forward_s3.json  shakib_Left_s6.json	shakib_Rest_s9.json
shakib_Backward_s1.json   shakib_Forward_s4.json  shakib_Left_s7.json	shakib_right_s10.json
shakib_Backward_s2.json   shakib_Forward_s5.json  shakib_Left_s8.json	shakib_right_s1.json
shakib_Backward_s3.json   shakib_Forward_s6.json  shakib_Left_s9.json	shakib_right_s2.json
shakib_Backward_s4.json   shakib_Forward_s7.json  shakib_Rest_s10.json	shakib_right_s3.json
shakib_Backward_s5.json   shakib_Forward_s8.json  shakib_Rest_s1.json	shakib_right_s4.json
shakib_Backward_s6.json   shakib_Forward_s9.json  shakib_Rest_s2.json	shakib_right_s5.json
shakib_Backward_s7.json   shakib_Left_s10.json	  shakib_Rest_s3.json	shakib_right_s6.json
shakib_Backward_s8.json   shakib_Left_s1.json	  shakib_Rest_s4.json	shakib_right_s7.json
shakib_Backward_s9.json   shakib_Left_s2.json	  shakib_Rest_s5.json	shakib_right_s8.json
shakib_Forward_s10.json   shakib_Left_s3.json	  shakib_Rest_s6.json	shakib_right_s9.json
shaki

In [ ]:
import os
import json
import numpy as np
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Bandpass filter
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return lfilter(b, a, data)

# Load and process raw data for LSTM
def load_raw_data(data_dir, fs=256, window_size=640, overlap=320):
    X, y = [], []
    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(data_dir, filename)
            with open(filepath, "r") as f:
                json_data = json.load(f)  # Load the JSON file (list of objects)

            combined_data = []  # Accumulate data for aggregation
            for obj in json_data:
                data = np.array(obj["data"])  # Shape: [channels, samples]
                combined_data.append(data)

                # If combined data has enough samples for a window
                if len(combined_data) * data.shape[1] >= window_size:
                    # Combine objects to form one window
                    segment = np.hstack(combined_data)[:, :window_size]  # [channels, window_size]
                    combined_data = []  # Reset for the next segment

                    # Bandpass filter the segment
                    filtered_segment = np.array([bandpass_filter(ch, 0.5, 40, fs) for ch in segment])
                    X.append(filtered_segment.T)  # Transpose to [window_size, channels]
                    y.append(filename.split("_")[1])  # Assign label based on filename
    return np.array(X), np.array(y)

# Load the data
data_dir = "/content/drive/My Drive/Shakib(10sec_each)"  # Directory containing your JSON files
X, y = load_raw_data(data_dir)

# Encode labels
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Build the LSTM Model


In [ ]:
model = Sequential([
    LSTM(64, input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True),
    Dropout(0.5),
    LSTM(64, return_sequences=False),
    Dropout(0.5),
    Dense(y_train.shape[1], activation="softmax")
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the LSTM model
model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=30, batch_size=32)

# Evaluate LSTM
y_pred = model.predict(X_test)
predicted_classes = np.argmax(y_pred, axis=1)
true_classes = np.argmax(y_test, axis=1)

print(f"Classification Report:\n{classification_report(true_classes, predicted_classes, target_names=le.classes_)}")


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


5/5 ━━━━━━━━━━━━━━━━━━━━ 9s 718ms/step - accuracy: 0.1771 - loss: 1.6754 - val_accuracy: 0.2000 - val_loss: 1.6558
Epoch 2/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 500ms/step - accuracy: 0.2938 - loss: 1.5990 - val_accuracy: 0.1750 - val_loss: 1.6632
Epoch 3/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 501ms/step - accuracy: 0.3140 - loss: 1.5572 - val_accuracy: 0.1750 - val_loss: 1.6734
Epoch 4/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 3s 677ms/step - accuracy: 0.3074 - loss: 1.5402 - val_accuracy: 0.1500 - val_loss: 1.6861
Epoch 5/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 774ms/step - accuracy: 0.2589 - loss: 1.5273 - val_accuracy: 0.1750 - val_loss: 1.6948
Epoch 6/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 526ms/step - accuracy: 0.4348 - loss: 1.4546 - val_accuracy: 0.1750 - val_loss: 1.7038
Epoch 7/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 5s 497ms/step - accuracy: 0.4368 - loss: 1.4552 - val_accuracy: 0.1750 - val_loss: 1.7174
Epoch 8/30
5/5 ━━━━━━━━━━━━━━━━━━━━ 4s 933ms/step - accuracy: 0.5025 - loss: 1.3886 - val_accuracy: 0.2000 - val_loss: 1.7238
Epo

In [21]:
import os
import json
import numpy as np
from scipy.signal import butter, lfilter, welch
from scipy.stats import skew, kurtosis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return lfilter(b, a, data)

# Feature extraction function
def extract_features(window, fs):
    freqs, psd = welch(window, fs, nperseg=fs // 2)
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 40)}
    band_powers = {band: np.sum(psd[(freqs >= low) & (freqs < high)]) for band, (low, high) in bands.items()}
    stats = {'mean': np.mean(window), 'variance': np.var(window), 'skewness': skew(window), 'kurtosis': kurtosis(window)}
    return {**band_powers, **stats}

def load_data_for_rf_and_lstm(data_dir, fs=256, window_size=1024, overlap=512):
    X_rf, X_lstm, y = [], [], []
    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(data_dir, filename)
            with open(filepath, "r") as f:
                json_data = json.load(f)  # Load the JSON file (list of objects)

            combined_data = []  # Accumulate data for aggregation
            for obj in json_data:
                data = np.array(obj["data"])  # Shape: [channels, samples]
                combined_data.append(data)

                # If combined data has enough samples for a window
                if len(combined_data) * data.shape[1] >= window_size:
                    # Combine objects to form one window
                    segment = np.hstack(combined_data)[:, :window_size]  # [channels, window_size]
                    combined_data = []  # Reset for the next segment

                    # Bandpass filter the segment
                    filtered_segment = np.array([bandpass_filter(ch, 0.5, 40, fs) for ch in segment])
                    X_lstm.append(filtered_segment.T)  # For LSTM, shape: [window_size, channels]

                    # Flatten the segment for Random Forest
                    X_rf.append(filtered_segment.flatten())  # For RF, shape: [1, channels * window_size]

                    # Assign label
                    y.append(filename.split("_")[1])  # Label from filename
    return np.array(X_rf), np.array(X_lstm), np.array(y)

# Load the data
data_dir = "/content/drive/My Drive/Shakib(10sec_each)/Forward"  # Directory containing your JSON files
X_rf, X_lstm, y = load_data_for_rf_and_lstm(data_dir)

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Split data into train and test sets
X_rf_train, X_rf_test, y_rf_train, y_rf_test = train_test_split(X_rf, y_encoded, test_size=0.2, random_state=42)
X_lstm_train, X_lstm_test, y_lstm_train, y_lstm_test = train_test_split(X_lstm, y_cat, test_size=0.2, random_state=42)

# Train a Random Forest Classifier
rf = RandomForestClassifier(n_estimators=100, random_state=42)
rf.fit(X_rf_train, y_rf_train)
y_rf_pred = rf.predict(X_rf_test)

print(f"Random Forest Accuracy: {accuracy_score(y_rf_test, y_rf_pred):.2f}")
print(classification_report(y_rf_test, y_rf_pred, target_names=le.classes_))



Random Forest Accuracy: 0.88
              precision    recall  f1-score   support

     Forward       1.00      0.80      0.89         5
        Rest       0.75      1.00      0.86         3

    accuracy                           0.88         8
   macro avg       0.88      0.90      0.87         8
weighted avg       0.91      0.88      0.88         8



ShallowNET

Enhanced Shallownet

ONLY LSTM

In [14]:
import os
import json
import numpy as np
from scipy.signal import butter, lfilter, welch
from scipy.stats import skew, kurtosis
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.utils import to_categorical

# Bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return lfilter(b, a, data)

# Feature extraction function
def extract_features(window, fs):
    freqs, psd = welch(window, fs, nperseg=fs // 2)
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 40)}
    band_powers = {band: np.sum(psd[(freqs >= low) & (freqs < high)]) for band, (low, high) in bands.items()}
    stats = {'mean': np.mean(window), 'variance': np.var(window), 'skewness': skew(window), 'kurtosis': kurtosis(window)}
    return {**band_powers, **stats}

def load_data_for_rf_and_lstm(data_dir, fs=256, window_size=1024, overlap=512):
    X_rf, X_lstm, y = [], [], []
    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(data_dir, filename)
            with open(filepath, "r") as f:
                json_data = json.load(f)  # Load the JSON file (list of objects)

            combined_data = []  # Accumulate data for aggregation
            for obj in json_data:
                data = np.array(obj["data"])  # Shape: [channels, samples]
                combined_data.append(data)

                # If combined data has enough samples for a window
                if len(combined_data) * data.shape[1] >= window_size:
                    # Combine objects to form one window
                    segment = np.hstack(combined_data)[:, :window_size]  # [channels, window_size]
                    combined_data = []  # Reset for the next segment

                    # Bandpass filter the segment
                    filtered_segment = np.array([bandpass_filter(ch, 0.5, 40, fs) for ch in segment])
                    X_lstm.append(filtered_segment.T)  # For LSTM, shape: [window_size, channels]

                    # Flatten the segment for Random Forest
                    X_rf.append(filtered_segment.flatten())  # For RF, shape: [1, channels * window_size]

                    # Assign label
                    y.append(filename.split("_")[1])  # Label from filename
    return np.array(X_rf), np.array(X_lstm), np.array(y)

# Load the data
data_dir = "/content/drive/My Drive/Shakib(10sec_each)/Forward"  # Directory containing your JSON files
X_rf, X_lstm, y = load_data_for_rf_and_lstm(data_dir)


In [15]:

# Encode labels
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y_encoded = le.fit_transform(y)
y_cat = to_categorical(y_encoded)

# Split data into train and test sets
X_rf_train, X_rf_test, y_rf_train, y_rf_test = train_test_split(X_rf, y_encoded, test_size=0.2, random_state=42)
X_lstm_train, X_lstm_test, y_lstm_train, y_lstm_test = train_test_split(X_lstm, y_cat, test_size=0.2, random_state=42)


In [ ]:
model = Sequential([
    LSTM(100, input_shape=(X_lstm_train.shape[1], X_lstm_train.shape[2]), return_sequences=True),
    Dropout(0.5),
    LSTM(100, return_sequences=False),
    Dropout(0.5),
    Dense(y_lstm_train.shape[1], activation="softmax")
])
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])

# Train the LSTM model
model.fit(X_lstm_train, y_lstm_train, validation_data=(X_lstm_test, y_lstm_test), epochs=30, batch_size=128)

# Evaluate LSTM
y_lstm_pred = model.predict(X_lstm_test)
predicted_classes = np.argmax(y_lstm_pred, axis=1)
true_classes = np.argmax(y_lstm_test, axis=1)

print(f"Classification Report:\n{classification_report(true_classes, predicted_classes, target_names=le.classes_)}")


Epoch 1/30


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


1/1 ━━━━━━━━━━━━━━━━━━━━ 7s 7s/step - accuracy: 0.5000 - loss: 0.6734 - val_accuracy: 0.2500 - val_loss: 0.8003
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.6562 - loss: 0.6054 - val_accuracy: 0.2500 - val_loss: 0.8075
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8125 - loss: 0.5672 - val_accuracy: 0.3750 - val_loss: 0.8244
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7500 - loss: 0.5429 - val_accuracy: 0.3750 - val_loss: 0.8562
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.7500 - loss: 0.5023 - val_accuracy: 0.3750 - val_loss: 0.9081
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - accuracy: 0.8438 - loss: 0.4360 - val_accuracy: 0.3750 - val_loss: 0.9718
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step - accuracy: 0.8750 - loss: 0.4068 - val_accuracy: 0.3750 - val_loss: 1.0395
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 1s/step - accuracy: 0.9688 - loss: 0.3337 - val_accuracy: 0.3750 - val_loss: 1.1165
Epoch 9/30
1/1 ━━━━━━━━━━━━

In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dropout, Dense, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2

# Build the enhanced LSTM model with more layers and higher units
def build_enhanced_lstm(input_shape, num_classes):
    model = Sequential([
        # First LSTM layer with more units
        LSTM(128, input_shape=input_shape, return_sequences=True),
        BatchNormalization(),
        Dropout(0.5),

        # Second LSTM layer with more units
        LSTM(128, return_sequences=True),
        BatchNormalization(),
        Dropout(0.5),

        # Third LSTM layer with more units
        LSTM(128, return_sequences=True),
        BatchNormalization(),
        Dropout(0.5),

        # Fourth LSTM layer with more units
        LSTM(128, return_sequences=True),
        BatchNormalization(),
        Dropout(0.5),

        # Fifth LSTM layer with more units
        LSTM(128, return_sequences=False),
        BatchNormalization(),
        Dropout(0.5),

        # Fully connected layer with more units
        Dense(128, activation='relu'),
        Dropout(0.5),

        # Output layer
        Dense(num_classes, activation='softmax')
    ])
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# Define model input shape and number of classes
input_shape_lstm = X_lstm_train.shape[1:]  # Shape: (time_steps, features)
num_classes = y_lstm_train.shape[1]

# Build the enhanced LSTM model
model_lstm = build_enhanced_lstm(input_shape_lstm, num_classes)

# Early stopping callback to prevent overfitting
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model
model_lstm.fit(
    X_lstm_train, y_lstm_train,
    validation_data=(X_lstm_test, y_lstm_test),
    epochs=30, batch_size=128,
    callbacks=[early_stopping]
)

# Evaluate the model
y_lstm_pred = model_lstm.predict(X_lstm_test)
predicted_classes = np.argmax(y_lstm_pred, axis=1)
true_classes = np.argmax(y_lstm_test, axis=1)

from sklearn.metrics import classification_report
print(f"Classification Report:\n{classification_report(true_classes, predicted_classes)}")


/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/rnn.py:204: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 24s 24s/step - accuracy: 0.5625 - loss: 1.3639 - val_accuracy: 0.1250 - val_loss: 0.7088
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 19s 19s/step - accuracy: 0.7188 - loss: 0.8008 - val_accuracy: 0.1250 - val_loss: 0.7105
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 18s 18s/step - accuracy: 0.5312 - loss: 1.1968 - val_accuracy: 0.1250 - val_loss: 0.7092
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 11s 11s/step - accuracy: 0.5625 - loss: 0.8767 - val_accuracy: 0.1250 - val_loss: 0.7098
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 20s 20s/step - accuracy: 0.6562 - loss: 0.8432 - val_accuracy: 0.1250 - val_loss: 0.7071
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 22s 22s/step - accuracy: 0.7188 - loss: 0.7502 - val_accuracy: 0.3750 - val_loss: 0.7033
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 12s 12s/step - accuracy: 0.6875 - loss: 0.6880 - val_accuracy: 0.3750 - val_loss: 0.6991
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 21s 21s/step - accuracy: 0.8125 - loss: 0.5941 - val_accuracy: 0.6250 - val_loss: 0.6919


CNN LSTM


In [24]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Conv1D, MaxPooling1D, LSTM, Dense, Dropout

# Build LRCN model
def build_lrcn(input_shape, num_classes):
    input_layer = Input(shape=input_shape)

    # CNN feature extractor
    x = Conv1D(filters=64, kernel_size=3, activation='relu')(input_layer)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)
    x = Conv1D(filters=128, kernel_size=3, activation='relu')(x)
    x = MaxPooling1D(pool_size=2)(x)
    x = Dropout(0.3)(x)

    # LSTM sequence modeler
    x = LSTM(64, return_sequences=False)(x)
    x = Dropout(0.5)(x)

    # Output layer
    output = Dense(num_classes, activation='softmax')(x)

    # Build model
    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Ensure input shape is correct: (time_steps, features)
input_shape_lrcn = (X_lstm_train.shape[1], X_lstm_train.shape[2])  # Shape: (time_steps, channels)

# Build and train the LRCN model
model_lrcn = build_lrcn(input_shape_lrcn, y_lstm_train.shape[1])
model_lrcn.fit(X_lstm_train, y_lstm_train, validation_data=(X_lstm_test, y_lstm_test), epochs=30, batch_size=32)

# Evaluate LRCN
loss_lrcn, accuracy_lrcn = model_lrcn.evaluate(X_lstm_test, y_lstm_test, verbose=0)
print(f"LRCN Test Accuracy: {accuracy_lrcn:.2f}")


Epoch 1/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 5s 5s/step - accuracy: 0.3750 - loss: 0.8547 - val_accuracy: 0.8750 - val_loss: 0.5837
Epoch 2/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 584ms/step - accuracy: 0.4688 - loss: 0.8257 - val_accuracy: 0.7500 - val_loss: 0.5947
Epoch 3/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 567ms/step - accuracy: 0.4375 - loss: 0.7896 - val_accuracy: 0.5000 - val_loss: 0.6025
Epoch 4/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 634ms/step - accuracy: 0.6562 - loss: 0.8346 - val_accuracy: 0.7500 - val_loss: 0.6245
Epoch 5/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 640ms/step - accuracy: 0.4375 - loss: 0.8245 - val_accuracy: 0.6250 - val_loss: 0.6081
Epoch 6/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 1s 635ms/step - accuracy: 0.4062 - loss: 0.8962 - val_accuracy: 0.6250 - val_loss: 0.6317
Epoch 7/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 373ms/step - accuracy: 0.6562 - loss: 0.6992 - val_accuracy: 0.5000 - val_loss: 0.6749
Epoch 8/30
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 316ms/step - accuracy: 0.5312 - loss: 0.7356 - val_accuracy: 0.5000 - val_loss: 0.

In [ ]:
import os
import numpy as np
from scipy.signal import butter, filtfilt, iirnotch, welch
from scipy.stats import skew, kurtosis, entropy
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from math import ceil

# Settings
SAMPLING_FREQ = 256
WINDOW_LEN = 1  # seconds
OVERLAP = 0.5
EPOCH_TIME = 0.0625  # seconds

channel_position_config = {
    'CP3': 0, 'C3': 1, 'F5': 2, 'PO3': 3,
    'PO4': 4, 'F6': 5, 'C4': 6, 'CP4': 7
}
electrode_config = {k: True for k in channel_position_config.keys()}
electrode_list = [channel_position_config[k] for k, v in electrode_config.items() if v]

feature_config = {
    "psd": True, "band_powers": True, "peak_frequency": True,
    "spectral_entropy": True, "mean_median_frequency": True,
    "bandwidth": True, "hjorth_parameters": True,
    "spectral_flatness": True, "statistical_features": True
}

preprocessing_config = {
    'bandpass': True, 'notch': True, 'baseline_correction': False,
    'z_score': True, 'ICA_artefacts': False, 'downsample': False
}

# ------------------------ Preprocessing Functions ------------------------
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low, high = lowcut / nyquist, highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, data)

def notch_filter(data, notch_freq, fs, quality_factor=30):
    nyquist = 0.5 * fs
    notch_freq = notch_freq / nyquist
    b, a = iirnotch(notch_freq, quality_factor)
    return filtfilt(b, a, data)

def zscore_normalization(data):
    mean, std = np.mean(data), np.std(data)
    return (data - mean) / std

def preprocess_eeg(data, fs):
    if preprocessing_config['bandpass']:
        data = bandpass_filter(data, lowcut=0.5, highcut=50.0, fs=fs, order=5)
    if preprocessing_config['notch']:
        data = notch_filter(data, notch_freq=60.0, fs=fs)
    if preprocessing_config['z_score']:
        data = zscore_normalization(data)
    return data

# ------------------------ Feature Extraction ------------------------
def hjorth_parameters(eeg_data):
    activity = np.var(eeg_data)
    mobility = np.sqrt(np.var(np.diff(eeg_data)) / activity)
    complexity = np.sqrt(np.var(np.diff(np.diff(eeg_data))) / np.var(np.diff(eeg_data))) / mobility
    return activity, mobility, complexity

def extract_features_eeg(eeg_data, fs):
    features = []
    freqs, psd = welch(eeg_data, fs, nperseg=fs*2)
    bands = {'delta': (0.5, 4), 'theta': (4, 8), 'alpha': (8, 13), 'beta': (13, 30), 'gamma': (30, 50)}
    for band, (low, high) in bands.items():
        band_power = np.trapz(psd[(freqs >= low) & (freqs <= high)], freqs[(freqs >= low) & (freqs <= high)])
        features.append(band_power)
    activity, mobility, complexity = hjorth_parameters(eeg_data)
    features.extend([activity, mobility, complexity])
    return np.array(features)

def extract_features_multi_channel(eeg_data_multi_channel, fs):
    features = []
    for channel_data in eeg_data_multi_channel:
        channel_data = preprocess_eeg(channel_data, fs)
        channel_features = extract_features_eeg(channel_data, fs)
        features.extend(channel_features)
    return np.array(features)

# ------------------------ Windowing Function ------------------------
def apply_window(win_len, overlap, eeg_data):
    total_samples = eeg_data.shape[1]
    step_size = int(win_len * SAMPLING_FREQ * (1 - overlap))
    num_windows = (total_samples - int(win_len * SAMPLING_FREQ)) // step_size + 1
    windows = []
    for start_idx in range(0, num_windows * step_size, step_size):
        window = eeg_data[:, start_idx:start_idx + int(win_len * SAMPLING_FREQ)]
        windows.append(window)
    return windows

# ------------------------ Main Workflow ------------------------
# Load and process all files in the directory
import json
def load_eeg_data(data_dir):
    all_features = []
    labels = []
    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(data_dir, filename)
            # Load JSON data
            with open(filepath, 'r') as f:
                json_data = json.load(f)
                # Extract "data" from each object in the JSON array
                eeg_data = [np.array(obj["data"]) for obj in json_data]
                eeg_data = np.hstack(eeg_data)  # Combine all segments into one array

            # Apply windowing
            windows = apply_window(WINDOW_LEN, OVERLAP, eeg_data)
            # Extract features for each window
            features = [extract_features_multi_channel(win, SAMPLING_FREQ) for win in windows]
            all_features.extend(features)
            # Assign label based on filename
            label = 0 if "Forward" in filename else 1
            labels.extend([label] * len(features))
    return np.array(all_features), np.array(labels)

forward_dir = "/content/drive/My Drive/Shakib(10sec_each)/Forward"
rest_dir = "/content/drive/My Drive/Shakib(10sec_each)/Rest"

# Load data for both classes
X_forward, y_forward = load_eeg_data(forward_dir)
X_rest, y_rest = load_eeg_data(rest_dir)

# Combine the data
X = np.vstack((X_forward, X_rest))
y = np.hstack((y_forward, y_rest))

# Split into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Train a Random Forest classifier
rf_clf = RandomForestClassifier(n_estimators=1000, random_state=42)
rf_clf.fit(X_train, y_train)

# Evaluate the classifier
y_pred = rf_clf.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Random Forest Accuracy: {accuracy:.2f}")
print("Classification Report:\n", classification_report(y_test, y_pred))


/usr/local/lib/python3.10/dist-packages/scipy/signal/_spectral_py.py:600: UserWarning: nperseg = 512 is greater than input length  = 256, using nperseg = 256
  freqs, _, Pxy = _spectral_helper(x, y, fs, window, nperseg, noverlap,


Random Forest Accuracy: 0.63
Classification Report:
               precision    recall  f1-score   support

           0       0.69      0.51      0.59        39
           1       0.60      0.76      0.67        37

    accuracy                           0.63        76
   macro avg       0.64      0.63      0.63        76
weighted avg       0.64      0.63      0.63        76



In [5]:
!pip install transformers torch torchvision

In [1]:
import os
import json
import numpy as np
from scipy.signal import butter, lfilter
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from tensorflow.keras.utils import to_categorical

# Bandpass filter function
def bandpass_filter(data, lowcut, highcut, fs, order=5):
    nyquist = 0.5 * fs
    low, high = lowcut / nyquist, highcut / nyquist
    b, a = butter(order, [low, high], btype="band")
    return lfilter(b, a, data)

# Load EEG data
def load_eeg_data(data_dir, class_label, fs=256, window_size=1024, overlap=512):
    X, y = [], []
    for filename in os.listdir(data_dir):
        if filename.endswith(".json"):
            filepath = os.path.join(data_dir, filename)
            with open(filepath, "r") as f:
                json_data = json.load(f)

            combined_data = []
            for obj in json_data:
                data = np.array(obj["data"])  # Shape: [channels, samples]
                combined_data.append(data)

                # If combined data has enough samples for a window
                if len(combined_data) * data.shape[1] >= window_size:
                    segment = np.hstack(combined_data)[:, :window_size]
                    combined_data = []

                    # Bandpass filter the segment
                    filtered_segment = np.array([bandpass_filter(ch, 0.5, 40, fs) for ch in segment])
                    X.append(filtered_segment.T)  # Shape: [window_size, channels]

                    # Assign label
                    y.append(class_label)
    return np.array(X), np.array(y)

# Paths to the directories
forward_dir = "/content/drive/My Drive/Shakib(10sec_each)/Forward"
rest_dir = "/content/drive/My Drive/Shakib(10sec_each)/Rest"

# Load data for both classes
X_forward, y_forward = load_eeg_data(forward_dir, class_label=0)  # 0 for Forward
X_rest, y_rest = load_eeg_data(rest_dir, class_label=1)            # 1 for Rest

# Combine the data
X = np.vstack((X_forward, X_rest))
y = np.hstack((y_forward, y_rest))

# Encode labels if necessary (not needed since labels are already numeric)
y_cat = to_categorical(y)  # Convert to one-hot encoding for classification

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y_cat, test_size=0.2, random_state=42)

# Debugging info
print("Shape of X_train:", X_train.shape)
print("Shape of y_train:", y_train.shape)
print("Class distribution:", np.sum(y_train, axis=0))


Shape of X_train: (32, 1024, 8)
Shape of y_train: (32, 2)
Class distribution: [15. 17.]


In [26]:
class TransformerBlock(Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(TransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([Dense(ff_dim, activation="relu"), Dense(embed_dim)])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = tf.keras.layers.Dropout(rate)
        self.dropout2 = tf.keras.layers.Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)


def build_transformer(input_shape, num_classes, embed_dim=64, num_heads=2, ff_dim=128):
    inputs = tf.keras.Input(shape=input_shape)
    x = Dense(embed_dim)(inputs)  # Project input to embedding dimension
    transformer_block = TransformerBlock(embed_dim, num_heads, ff_dim)
    x = transformer_block(x)  # First Transformer block
    x = transformer_block(x)  # Second Transformer block
    x = tf.keras.layers.GlobalAveragePooling1D()(x)
    x = Dense(128, activation="relu")(x)
    x = tf.keras.layers.Dropout(0.1)(x)
    outputs = Dense(num_classes, activation="softmax")(x)
    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
    return model

input_shape = (X_train.shape[1], X_train.shape[2])  # (window_size, channels)
num_classes = y_train.shape[1]
transformer_model = build_transformer(input_shape, num_classes)

# Train the model
transformer_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=10, batch_size=32)

# Evaluate the model
loss, accuracy = transformer_model.evaluate(X_test, y_test, verbose=0)
print(f"EEG Transformer Test Accuracy: {accuracy:.2f}")

In [28]:
from tensorflow.keras.layers import Conv1D, BatchNormalization, Input, Dense, GlobalAveragePooling1D, Dropout, LayerNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam

# Improved Transformer Block
class EnhancedTransformerBlock(tf.keras.layers.Layer):
    def __init__(self, embed_dim, num_heads, ff_dim, rate=0.1):
        super(EnhancedTransformerBlock, self).__init__()
        self.att = MultiHeadAttention(num_heads=num_heads, key_dim=embed_dim)
        self.ffn = Sequential([
            Dense(ff_dim, activation="relu"),
            Dense(embed_dim)
        ])
        self.layernorm1 = LayerNormalization(epsilon=1e-6)
        self.layernorm2 = LayerNormalization(epsilon=1e-6)
        self.dropout1 = Dropout(rate)
        self.dropout2 = Dropout(rate)

    def call(self, inputs, training=None):
        attn_output = self.att(inputs, inputs, training=training)
        attn_output = self.dropout1(attn_output, training=training)
        out1 = self.layernorm1(inputs + attn_output)
        ffn_output = self.ffn(out1, training=training)
        ffn_output = self.dropout2(ffn_output, training=training)
        return self.layernorm2(out1 + ffn_output)

def build_advanced_transformer(input_shape, num_classes, embed_dim=64, num_heads=4, ff_dim=128, num_blocks=4):
    inputs = Input(shape=input_shape)

    # Convolutional Block
    x = Conv1D(64, kernel_size=3, activation='relu', padding='same')(inputs)
    x = BatchNormalization()(x)
    x = Conv1D(128, kernel_size=3, activation='relu', padding='same')(x)
    x = BatchNormalization()(x)

    # Positional Encoding
    x = Dense(embed_dim)(x)
    for _ in range(num_blocks):
        x = EnhancedTransformerBlock(embed_dim, num_heads, ff_dim)(x)

    # Pooling and Dense Layers
    x = GlobalAveragePooling1D()(x)
    x = Dense(256, activation="relu")(x)
    x = Dropout(0.2)(x)
    outputs = Dense(num_classes, activation="softmax")(x)

    model = Model(inputs=inputs, outputs=outputs)
    model.compile(optimizer=Adam(learning_rate=1e-3), loss="categorical_crossentropy", metrics=["accuracy"])
    return model

# Define and Train the Enhanced Model
advanced_transformer_model = build_advanced_transformer(input_shape, num_classes)
advanced_transformer_model.fit(X_train, y_train, validation_data=(X_test, y_test), epochs=20, batch_size=64)

# Evaluate Model
loss, accuracy = advanced_transformer_model.evaluate(X_test, y_test)
print(f"Improved EEG Transformer Test Accuracy: {accuracy:.2f}")


Epoch 1/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 58s 58s/step - accuracy: 0.5312 - loss: 0.8602 - val_accuracy: 0.3750 - val_loss: 0.6980
Epoch 2/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 0.4688 - loss: 0.8160 - val_accuracy: 0.2500 - val_loss: 0.6947
Epoch 3/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 39s 39s/step - accuracy: 0.4688 - loss: 0.7240 - val_accuracy: 0.3750 - val_loss: 0.7295
Epoch 4/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 0.5938 - loss: 0.7105 - val_accuracy: 0.3750 - val_loss: 0.7219
Epoch 5/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 34s 34s/step - accuracy: 0.5312 - loss: 0.7104 - val_accuracy: 0.5000 - val_loss: 0.6934
Epoch 6/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 36s 36s/step - accuracy: 0.6562 - loss: 0.6519 - val_accuracy: 0.6250 - val_loss: 0.6785
Epoch 7/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 38s 38s/step - accuracy: 0.5938 - loss: 0.6907 - val_accuracy: 0.6250 - val_loss: 0.6776
Epoch 8/20
1/1 ━━━━━━━━━━━━━━━━━━━━ 41s 41s/step - accuracy: 0.5625 - loss: 0.6714 - val_accuracy: 0.6250 - val_loss: 0.6847
